#### Prj04

Consider the following data sets:

- Today: Sep 27, 2018
- Spot price: 290.68
- Maturity: Dec 31, 2018
- Strike: 288
- rate: 0.02 (from https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yield)
- Market call price: 9.23

###### Todo

If the volatility is $30\%$, what is the difference between BSM call price and market call price?
Compute the implied volatility.

In [70]:
import numpy as np
import scipy.stats as ss
import time 
import datetime
import math
import scipy.optimize as sop
from dateutil.parser import parse

We define the functions of BSM for European call and put, based on  
$$C_0 = \mathbb E [e^{-rT} (S(T) - K)^+] = S_0  \Phi(d_1) - K e^{-rT} \Phi(d_2)$$
$$P_0 = \mathbb E [e^{-rT} (S(T) - K)^-] = K e^{-rT} \Phi(- d_2) - S_0  \Phi(- d_1)$$

In [52]:
def BSM_call_option(St, K , t, T, r, sigma):

    '''
    Parameters
    ==========
    St : float
        stock/index level at time t
    K : float
        strike price
    t : float
        valuation date
    T : float
        date of maturity/time-to-maturity if t = 0; T > t
    r : float
        constant, risk-less short rate
    sigma : float
        volatility
    =======
    '''

    d1 = (math.log(St/K) + (r+0.5*pow(sigma,2)) * (T - t)) / (sigma * math.sqrt(T - t))
    d2 = (math.log(St/K) + (r-0.5*pow(sigma,2)) * (T - t)) / (sigma * math.sqrt(T - t))
    call_value = St * ss.norm.cdf(d1) - math.exp(-r * (T - t)) * K * ss.norm.cdf(d2)
    return call_value

In [59]:
def BSM_put_option(St, K, t, T, r, sigma):   
    d1 = (math.log(St/K) + (r+.5*pow(sigma,2)) * (T - t)) / (sigma * math.sqrt(T - t))
    d2 = (math.log(St/K) + (r-.5*pow(sigma,2)) * (T - t)) / (sigma * math.sqrt(T - t))
    put_value = -St * ss.norm.cdf(-d1) + math.exp(-r * (T - t)) * K * ss.norm.cdf(-d2)
    return put_value

Then, we calculate the European call option value based on the given parameters:

In [61]:
dt = parse('Dec 31, 2018') - parse('sep 27, 2018')
t = 0
T = dt.days/365
S0 = 290.68
r = 0.02
K = 288
C0 = 9.23
sigma = 0.3

In [67]:
call_value = BSM_call_option(S0, K, t, T, r, sigma)
print('The BSM call price is {0:5.3f}, The difference between BSM and market price is {1:5.3f}' .format(call_value, call_value-C0))

The BSM call price is 19.770, The difference between BSM and market price is 10.540


Then, we define an error function for iteratively calculating the volatility.

In [73]:
def BSM_error_function(sigma):
    MSE = np.abs(BSM_call_option(S0, K, t, T, r, sigma) - C0)
    return MSE

Volatility = sop.fmin(BSM_error_function, 0.3, xtol = 0.0001, ftol = 0.0001)

print("The implied volatility is", Volatility)

Optimization terminated successfully.
         Current function value: 0.000373
         Iterations: 17
         Function evaluations: 34
The implied volatility is [0.11822754]
